In [17]:
import pandas as pd
import numpy as np
import random

In [18]:
def cargar_ficheros():
    
    onlyfiles = [f for f in listdir('Datos') if isfile(join('Datos', f))]

    datos = []

    for fichero in onlyfiles:

         
        df = pd.read_csv(join('Datos', fichero))    
        datos.append(df)

    datos = pd.concat(datos)
    return datos

In [19]:
def comprobar_homogeneizacion(datos):

 

    long_fechas_unicas = datos.date.unique().shape[0]
    long_isin_unicos = datos.iloc[:,2].unique().shape[0]

    if (long_fechas_unicas * long_isin_unicos == datos.shape[1]) == True:
        print("Los activos están homogeneizados")
        return True
    else:
        print("Los activos NO están homogeneizados")
        return False    
    

In [20]:
def extraccion_datos(isin, datos):
    
   
    df_intermedio = datos.loc[datos["isin"]==isin,:]

   
    nuevo_df = pd.DataFrame(df_intermedio, columns=["date","nav"])
    

    nuevo_df = nuevo_df.groupby('date').first().reset_index()      
    
    nuevo_df.index = nuevo_df.date  
    nuevo_df.index = pd.to_datetime(nuevo_df.index)    
    del nuevo_df["date"]
    nuevo_df.columns = [isin]
    
    return nuevo_df

In [21]:
def homogeizar(datos):

    lista_isin = datos.iloc[:,2].unique()

    fechas_posibles = datos.date.unique()
    fechas_posibles.sort()
    fechas_posibles = pd.date_range(fechas_posibles[0], fechas_posibles[-1], freq=BDay())
    datos_ordenados = pd.DataFrame(columns=[""], index = fechas_posibles) 

    for isin in lista_isin:

        print(isin)
        isin_extraido = extraccion_datos(isin, datos)

       
        if (isin_extraido.shape[0] > fechas_posibles.shape[0]*0.7):

            datos_ordenados = pd.concat([datos_ordenados, isin_extraido], axis=1) 

    datos_ordenados = datos_ordenados.drop(columns='')


    dias_eliminar = ~(datos_ordenados.isna().sum(axis=1) > datos_ordenados.shape[1]*0.9)
    datos_ordenados = datos_ordenados.loc[dias_eliminar,:]


    datos_ordenados = datos_ordenados.fillna(method='ffill')

  
    datos_ordenados = datos_ordenados.fillna(method='bfill')

    datos_ordenados.to_pickle("datos_ordenados.pkl")     
    
    return datos_ordenados

In [22]:
def cargar_datos():

    try:
        datos_ordenados = pd.read_pickle("datos_ordenados.pkl")

    except:
        datos = cargar_ficheros()
        comprobacion = comprobar_homogeneizacion(datos)

        if comprobacion == False:
            datos_ordenados = homogeizar(datos)
            
    return datos_ordenados

## Generamos la población inicial

In [23]:
def asignar_pesos(posicion_cartera, num_sim = 100):

    

    pesos_cartera = np.empty([num_sim, len(posicion_cartera)])

    for sim in range(num_sim):

      
        posiciones_aleatorias = random.sample(range(len(posicion_cartera)), len(posicion_cartera))
        peso_total = 0

        for pos in posiciones_aleatorias:

            if peso_total < 1:

                aleatorio = np.random.random()        

                pesos_cartera[sim, pos] = aleatorio
                peso_total += pesos_cartera[sim, pos]

                if peso_total >= 1:

                    exceso = peso_total - 1
                    pesos_cartera[sim, pos] = pesos_cartera[sim, pos] - exceso
                    peso_total = 1

            else:

                
                pesos_cartera[sim, pos] = 0

        
        if pesos_cartera[sim,:].sum() < 1:

            pesos_cartera[sim,:] = pesos_cartera[sim,:] / pesos_cartera[sim,:].sum()

    return pesos_cartera

In [24]:
def generar_poblacion_inicial(datos_ordenados, num_act_min = 2, num_act_max = 20):

   

    num_inversores = int(np.ceil((datos_ordenados.shape[1] * 50) / (5 + datos_ordenados.shape[1])))
    num_activos_gen_inicial = np.random.randint(low=num_act_min, high=num_act_max, size=num_inversores)
    lista_posiciones = list()
    
    for inv in range(num_inversores):

        
        
        posicion_cartera = np.random.randint(low=0, high= (datos_ordenados.shape[1]-1), size=num_activos_gen_inicial[inv])
        lista_posiciones.append(posicion_cartera)
           
    return lista_posiciones


## Calculamos la función de fitness

In [25]:
def calcular_fitness(datos_ordenados, posiciones):
    
    
    
    datos = datos_ordenados.iloc[:,list(posiciones)]

    # Calculamos la rentabilidad de los activos
    rent_activos = np.log(datos).diff() 
    rent_activos = rent_activos.iloc[1:,:]

    
    matriz_correlaciones = rent_activos.corr()

    
    matriz_covarianzas = rent_activos.cov()
    matriz_covarianzas = matriz_covarianzas.to_numpy(dtype='float') 


    precio_inicial = datos.iloc[0,:]
    precio_final = datos.iloc[-1,:]
    rentabilidad_diaria = np.log(precio_final / precio_inicial) / datos.shape[0]

    # Sacamos la matriz de pesos para cada cartera
    matriz_pesos = asignar_pesos(posiciones)
    
    
    auxiliar = rentabilidad_diaria.values * matriz_pesos
    rentabilidad_cartera = auxiliar.sum(axis=1)

   
    auxiliar = np.dot(matriz_pesos, matriz_covarianzas) # Reutilizo la matriz auxiliar con otro propósito para no sobrecargar la RAM
    riesgo_cartera = pow((auxiliar * matriz_pesos).sum(axis=1), 0.5)

    
    posiciones_0entre0 = (rentabilidad_cartera==0) * (riesgo_cartera==0)    
    riesgo_cartera[posiciones_0entre0] = 0.0000001
    eficiencia_cartera = rentabilidad_cartera / riesgo_cartera
    
    
    posicion_cartera_eficiente = np.where(eficiencia_cartera == eficiencia_cartera.max())
    eficiencia_cartera = eficiencia_cartera.max()
    pesos_cartera_eficiente = matriz_pesos[posicion_cartera_eficiente,:]
        
    return eficiencia_cartera, pesos_cartera_eficiente

## Selección de padres

In [26]:
def seleccion_padres(lista_fitness):

    
    fitness_normalizado = np.asarray(lista_fitness)
    fitness_normalizado = (fitness_normalizado - fitness_normalizado.min()) / (fitness_normalizado.max() - fitness_normalizado.min())

    
    orden_aleatorio = random.sample(range(len(fitness_normalizado)), len(fitness_normalizado))

    padres_seleccionados = list()
    for candidato in orden_aleatorio:

        
        aleatorio = np.random.random() 

        if fitness_normalizado[candidato] > aleatorio:

            padres_seleccionados.append(candidato)

            if len(padres_seleccionados) == 2:

                break
   
    return padres_seleccionados

## Cruzamiento y mutación

In [27]:
def cruzamiento(datos_ordenados, lista_posiciones, lista_pesos_carteras_fitness, padres_seleccionados, umbral = 0.4, num_act_max = 20):

    '''
    Las técnicas habituales de cruzamiento en algoritmos genéticos (N-puntos, segmentación, uniforme y aleatorio) no parecen apropiadas para resolver este problema
    Queremos generar un sitema que cruce la información genética de los padres, permitiendo que el tamaño (número de activos) del hijo difiera del de los padres (sin tener que ser forzosamente mayor)
    EL sistema de cruzamiento debe centrarse en la selección de activos y no en los pesos. Dado que los pesos serán calculados durante la función de fitness
    Hemos usado en las funciones previas dos de los elementos más importantes para resolver el problema: pesos y eficiencia (relación rentabilidad riesgo)
    Para hacer el cruzamiento, debemos usar el último elemento importante que queda: la correlación (medida normalizada de la covarianza)

    '''

   
    posiciones_activos_con_peso_papa = np.where(lista_pesos_carteras_fitness[padres_seleccionados[0]] != 0)
    posiciones_activos_con_peso_mama = np.where(lista_pesos_carteras_fitness[padres_seleccionados[1]] != 0)

    activos_con_peso_papa = lista_posiciones[padres_seleccionados[0]][posiciones_activos_con_peso_papa[2]]
    activos_con_peso_mama = lista_posiciones[padres_seleccionados[1]][posiciones_activos_con_peso_mama[2]]


    activos_con_peso = np.unique(np.append(activos_con_peso_papa, activos_con_peso_mama))
    activos_con_peso

    
    datos = datos_ordenados.iloc[:,activos_con_peso]

    precio_inicial = datos.iloc[0,:]
    precio_final = datos.iloc[-1,:]
    rent_global_activos = np.log(precio_final / precio_inicial) 

    riesgo_global_activos = np.std(datos, axis=0)

    eficiencia_global_activos = rent_global_activos / riesgo_global_activos

  
    rent_activos = np.log(datos).diff() 
    rent_activos = rent_activos.iloc[1:,:]

    matriz_correlaciones = rent_activos.corr()

   

    relaciones_a_analizar = matriz_correlaciones[np.logical_and(matriz_correlaciones > umbral, matriz_correlaciones != 1)]
    relaciones_a_analizar = np.where(matriz_correlaciones == relaciones_a_analizar)

    if len(relaciones_a_analizar[0]) > 0:

       
        activos_a_eliminar = list()

        for pos in range(len(relaciones_a_analizar[0])):

            if eficiencia_global_activos.iloc[relaciones_a_analizar[0][pos]] > eficiencia_global_activos.iloc[relaciones_a_analizar[1][pos]]:

                activos_a_eliminar.append(relaciones_a_analizar[1][pos])

            else:

                activos_a_eliminar.append(relaciones_a_analizar[0][pos])

        activos_a_eliminar = list(set(activos_a_eliminar))    
        activos_con_peso = np.delete(activos_con_peso, activos_a_eliminar)

    
    nuevos_activos = mutacion(datos_ordenados, activos_con_peso, num_act_max = 20)

    activos_heredados = np.unique(np.append(activos_con_peso, nuevos_activos))

   
    if len(activos_heredados) > num_act_max:

        seleccion_aleatoria = random.sample(range(len(activos_heredados)), len(activos_heredados))[0:20]
        activos_heredados = activos_heredados[seleccion_aleatoria]    

    return activos_heredados

In [28]:
def mutacion(datos_ordenados, activos_con_peso, num_act_max = 20):
    
   

    num_nuevos_activos = np.random.randint(low=1, high=4, size=1)

    nuevos_activos = np.random.randint(low=0, high=datos_ordenados.shape[1]-1, size=num_nuevos_activos)

    return nuevos_activos

## Reemplazo generacional

In [34]:
def reemplazo_generacional(datos_ordenados, lista_posiciones, lista_fitness, lista_pesos_carteras_fitness, activos_cartera_mejor_individuo):

    '''
    La estrategia de reemplazo generacional que vamos a utilizar es reemplazo total. 
    La generación de hijos reemplaza a sus padres, a excepción del mejor de la generación anterior.
    '''
    
    
    matriz_descendencia = list()
    matriz_descendencia.append(activos_cartera_mejor_individuo)

    for hijo in range(len(lista_posiciones)-1):

    
        padres_seleccionados = seleccion_padres(lista_fitness)

        
        activos_heredados = cruzamiento(datos_ordenados, lista_posiciones, lista_pesos_carteras_fitness, padres_seleccionados, umbral = 0.4, num_act_max = 20)

       
        matriz_descendencia.append(activos_heredados)

    #la matriz de descendencia se ha de llamar lista_posiciones
    lista_posiciones = matriz_descendencia

    return lista_posiciones

## Ejecutamos el algoritmo genético

In [35]:
datos_ordenados = cargar_datos()

# Generamos la población inicial
lista_posiciones = generar_poblacion_inicial(datos_ordenados, num_act_min = 2, num_act_max = 20)
fitness_mejor_individuo = 0
generaciones_sin_mejora = 0
generacion = 0

while generaciones_sin_mejora < 100:

    generacion += 1

    # Calculamos la función de fitness a la generación actual
    lista_fitness = list()
    lista_pesos_carteras_fitness =  list()
    for cartera in range(len(lista_posiciones)):

        fitness, pesos_fitness = calcular_fitness(datos_ordenados = datos_ordenados, posiciones = lista_posiciones[cartera])
        lista_fitness.append(fitness)
        lista_pesos_carteras_fitness.append(pesos_fitness)

    # Guardamos el mejor individuo de la generación actual
    if fitness_mejor_individuo < max(lista_fitness):

        fitness_mejor_individuo = max(lista_fitness)            
        posicion_mejor_individuo = np.where(lista_fitness == fitness_mejor_individuo)
        pesos_cartera_mejor_individuo = lista_pesos_carteras_fitness[posicion_mejor_individuo[0][0]]
        activos_cartera_mejor_individuo = lista_posiciones[posicion_mejor_individuo[0][0]]
        print(f'Generación {generacion}. El mejor individuo tiene una eficiencia de {fitness_mejor_individuo}')
        generaciones_sin_mejora = 0

    else:

        print(f'Generación {generacion+1}. Sin mejora generacional respecto a la anterior')
        generaciones_sin_mejora += 1

    # Llevamos a cabo la selección de padres, cruzamiento, mutación y matriz de descendencia
    lista_posiciones = reemplazo_generacional(datos_ordenados, lista_posiciones, lista_fitness, lista_pesos_carteras_fitness, activos_cartera_mejor_individuo)

print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)    

Generación 1. El mejor individuo tiene una eficiencia de 0.3234185587483984
Generación 2. El mejor individuo tiene una eficiencia de 0.3939207860840887
Generación 3. El mejor individuo tiene una eficiencia de 0.42228106989080594
Generación 4. El mejor individuo tiene una eficiencia de 0.4438165797570238
Generación 6. Sin mejora generacional respecto a la anterior
Generación 7. Sin mejora generacional respecto a la anterior
Generación 8. Sin mejora generacional respecto a la anterior
Generación 9. Sin mejora generacional respecto a la anterior
Generación 9. El mejor individuo tiene una eficiencia de 0.4655786460113092
Generación 10. El mejor individuo tiene una eficiencia de 0.4719314530172421
Generación 11. El mejor individuo tiene una eficiencia de 0.48287069829374046
Generación 12. El mejor individuo tiene una eficiencia de 0.4867779341017799
Generación 14. Sin mejora generacional respecto a la anterior
Generación 14. El mejor individuo tiene una eficiencia de 0.5121176667044909
Gene

In [36]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.2873223109557326
[22808 32953 39048 39070 48011 56525]
[[[0.06846977 0.         0.14204205 0.5223331  0.26715509 0.        ]]]


In [37]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.2873223109557326
[22808 32953 39048 39070 48011 56525]
[[[0.06846977 0.         0.14204205 0.5223331  0.26715509 0.        ]]]


In [38]:
print(fitness_mejor_individuo)
print(activos_cartera_mejor_individuo)
print(pesos_cartera_mejor_individuo)

1.2873223109557326
[22808 32953 39048 39070 48011 56525]
[[[0.06846977 0.         0.14204205 0.5223331  0.26715509 0.        ]]]
